# Reverse-engineering of the surron ultrabee can bus

In [14]:
import pandas as pd
import matplotlib.pyplot as plt
import csv
import json

## Reading all CSV Files

In [16]:

with open('20240427_CAN_Trace_Motorrad_T000.csv', 'r') as f:
    reader = csv.reader(f, delimiter=";")
    data00 = list(reader)

with open('20240427_CAN_Trace_Motorrad_T001.csv', 'r') as f:
    reader = csv.reader(f, delimiter=";")
    data01 = list(reader)

data00.pop(0)
data01.pop(0)

dataCombined = data00 + data01

df = pd.DataFrame(dataCombined)
df


,0,1,2,3,4
0,0.008923,Rx,8,00 00 00 00 02 00 00 02,18
1,0.017042,Rx,8,25 03 00 00 00 00 00 28,258
2,0.017292,Rx,8,00 00 00 00 00 00 00 00,259
3,0.017540,Rx,8,B2 20 00 00 00 00 00 D2,260
4,0.017788,Rx,8,10 27 00 00 00 00 00 37,261
...,...,...,...,...,...
55881,39.316.511,Rx,8,FF FF 2C 04 5A 00 00 88,259
55882,39.316.759,Rx,8,B2 20 00 00 00 00 00 D2,260
55883,39.317.007,Rx,8,10 27 00 00 00 00 00 37,261
55884,39.317.296,Rx,8,00 03 E7 03 2A 00 C0 0F,417333248x


## Extracting all Ids 

In [20]:
messengeIds = list(map(lambda row: (row[4]), data00))
uniqueIds = set(messengeIds)

df = pd.DataFrame(uniqueIds)
df

,0
0,516
1,788
2,417333504x
3,517
4,791
5,275
6,417333259x
7,801
8,771
9,792


## Massage CNT per Id

## Extracting specific ids

In [13]:
data00Filtered = list(filter(lambda row: (row[4] == "417333248x"), data00))

df = pd.DataFrame(data00Filtered)
df

,0,1,2,3,4
0,0.018079,Rx,8,00 07 E7 03 9C 0F C2 0F,417333248x
1,0.043026,Rx,8,00 07 E7 03 AD 0F C5 0F,417333248x
2,0.058161,Rx,8,00 07 E7 03 C4 0F C5 0F,417333248x
3,0.078231,Rx,8,00 07 E7 03 CE 0F C2 0F,417333248x
4,0.098286,Rx,8,00 07 E7 03 E0 0F C2 0F,417333248x
...,...,...,...,...,...
4806,106.479.729,Rx,8,00 03 E7 03 2A 00 C0 0F,417333248x
4807,106.499.987,Rx,8,00 03 E7 03 28 00 C0 0F,417333248x
4808,106.519.796,Rx,8,00 03 E7 03 28 00 C0 0F,417333248x
4809,106.540.856,Rx,8,00 03 E7 03 28 00 C0 0F,417333248x
